In [2]:
from netCDF4 import Dataset
import numpy as np
import json
from netCDFfunc.utility import get_data_by_date, get_stat, get_data_A
import os
from tqdm.notebook import tqdm
import datetime as dt

In [3]:
dsin = Dataset(f'/Volumes/T7/AVHRR_OI_SST/v2.1/1981/oisst-avhrr-v02r01.19810901.nc', 'r', format='NETCDF4')

In [20]:
dsin.variables['lon'][:]

masked_array(data=[1.25000e-01, 3.75000e-01, 6.25000e-01, ...,
                   3.59375e+02, 3.59625e+02, 3.59875e+02],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [4]:
dsout = Dataset(f'/Volumes/T7/new_data/test.ncdf', 'w', format='NETCDF4')

In [21]:
dsin = Dataset(f'/Volumes/T7/other_data/oisst_ssta_monthly(1982-2019).nc', 'r', format='NETCDF4')

In [27]:
dsin.variables['avgsst'][:]

masked_array(
  data=[[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [-1.6829963, -1.6803886, -1.6780941, ..., -1.6916536,
          -1.688606 , -1.6857027],
         [-1.7147669, -1.713818 , -1.7129904, ..., -1.7180969,
          -1.7169539, -1.7158173],
         [-1.7307675, -1.7306178, -1.7305027, ..., -1.7312331,
          -1.7310793, -1.7309115]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [-1.6877996, -1.6856889, -1.6836358, ..., -1.6959026,
          -

In [6]:
dsin.dimensions

{'ntime': <class 'netCDF4._netCDF4.Dimension'>: name = 'ntime', size = 456,
 'nlat': <class 'netCDF4._netCDF4.Dimension'>: name = 'nlat', size = 180,
 'nlon': <class 'netCDF4._netCDF4.Dimension'>: name = 'nlon', size = 360,
 'nmon': <class 'netCDF4._netCDF4.Dimension'>: name = 'nmon', size = 12}

In [9]:
dsin.variables['sst'][:].data.shape

(456, 180, 360)

In [17]:
arr = dsin.variables['sst'][:].data

In [35]:
np.nanmax(arr)

33.739998

In [4]:
dsin.close()
dsout.close()

## copy data

In [ ]:
# set attribute
for attr in dsin.ncattrs() :
    dsout.setncattr(attr, dsin.getncattr(attr))

In [ ]:
# set dimension
for k, v in dsin.dimensions.items():
    dsout.createDimension(k, v.size)

In [6]:
# set variables
for name, variable in dsin.variables.items():
    
    new_variable = dsout.createVariable(name, variable.datatype, variable.dimensions, fill_value=-999) # name, datatype, dimension
    dsout[name][:] = dsin[name][:] # values
    
    for attr in variable.ncattrs():
        if attr == '_FillValue': continue
        new_variable.setncattr(attr, variable.getncattr(attr)) # variable attr

## write data

In [2]:
start_date = (1981,9,1)
end_date = (2011,8,31)
result = get_data_by_date(get_data_A, 'sst', start_date, end_date)

  0%|          | 0/31 [00:00<?, ?it/s]

In [3]:
mean = get_stat(result, 'mean')

0it [00:00, ?it/s]

In [4]:
perc = get_stat(result, 'perc')

0it [00:00, ?it/s]

In [5]:
cnt = get_stat(result, 'cnt')

0it [00:00, ?it/s]

In [7]:
def copy_existing_variable(dsin, dsout, default_fill_value):
    
    for attr in dsin.ncattrs() :
        
        if attr == 'anom' : continue
        elif attr == 'err' : new_attr = 'analysis_error'
        elif attr == 'ice' : new_attr = 'sea_ice_fraction'
        elif attr == 'sst' : new_attr = 'analysed_sst'
        else : new_attr = attr
        
        dsout.setncattr(new_attr, dsin.getncattr(attr))

    for k, v in dsin.dimensions.items():
        dsout.createDimension(k, v.size)
    
    for name, variable in dsin.variables.items():
    
        existing_variable = dsout.createVariable(name, variable.datatype, variable.dimensions, fill_value=default_fill_value) # name, datatype, dimension
        dsout[name][:] = dsin[name][:] # values

        for attr in variable.ncattrs():
            if attr == '_FillValue': continue
            existing_variable.setncattr(attr, variable.getncattr(attr)) # variable attr
            
    return dsout

def create_new_variable(dsout, new_variable_name, dtype, dimension, fill_value, values, attributes):
            
    new_variable = dsout.createVariable(new_variable_name, dtype, dimension, fill_value=fill_value)
    dsout[new_variable_name][:] = values
    
    for k,v in attributes.items():
        if k == '_FillValue': continue
        new_variable.setncattr(k, v) # variable attr
    
    return dsout

In [8]:
def transfer_data(year, month, date, variable_types, variable_values): 
    '''
    variable_type = 'mean' or 'perc' or 'cnt'
    '''
    
    date = dt.date(year,month,day).strftime('%Y%m%d')
    
    if year < 2016 : 
        dsin = Dataset(f'/Volumes/T7/AVHRR_OI_SST/v2.1/{year}/oisst-avhrr-v02r01.{date}.nc', 'r', format='NETCDF4')
    else :
        if os.path.exists(f'/Volumes/T7/AVHRR_OI_SST/v2.1/{year}/{date}120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.0-fv02.1.nc'):
            dsin = Dataset(f'/Volumes/T7/AVHRR_OI_SST/v2.1/{year}/{date}120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.0-fv02.1.nc', 'r', format='NETCDF4')
        else :
            dsin = Dataset(f'/Volumes/T7/AVHRR_OI_SST/v2.1/{year}/{date}120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.1-fv02.1.nc', 'r', format='NETCDF4')
    
    if not os.path.exists(f'/Volumes/T7/new_data/AVHRR_OI_SST/{year}'):
        os.makedirs((f'/Volumes/T7/new_data/AVHRR_OI_SST/{year}'))
                           
    dsout = Dataset(f'/Volumes/T7/new_data/AVHRR_OI_SST/{year}/{date}120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.1-fv02.1.nc', 'w', format='NETCDF4')
    
    dsout = copy_existing_variable(dsin, dsout, default_fill_value=-999)
    
    for variable_type, variable_value in zip(variable_types, variable_values):

        if variable_type == 'mean' :
            variable_name = 'mean_sst'
            dtype = np.float32
            attribute = {'long_name' : 'mean sst for 30 years (1981/9/1~2011/8/31)',
                         'add_offset' : 0.0,
                         'scale_factor' : 0.01,
                         'valid_min' : -1200,
                         'valid_max' : 1200,
                         'units' : 'celcius'}

        if variable_type == 'perc' :
            variable_name = '90-percentile_sst'
            dtype = np.float32
            attribute = {'long_name' : '90-percentile of sst for 30years (1981/9/1~2011/8/31)',
                         'add_offset' : 0.0,
                         'scale_factor' : 0.01,
                         'valid_min' : -1200,
                         'valid_max' : 1200,
                         'units' : 'celcius'}

        if variable_type == 'cnt' :
            variable_name = 'cnt_of_value'
            dtype = np.int16
            attribute = {'long_name' : 'based values counted by pixel (30,0 = normal, other = anormal)',
                         'add_offset' : 0,
                         'scale_factor' : 1,
                         'valid_min' : 0,
                         'valid_max' : 30,
                         'units' : '-'}

        # common attribute
        dimensions = ('time', 'zlev', 'lat', 'lon')
        fill_value = -999
        values = variable_value[(month,day)]

        dsout = create_new_variable(dsout, variable_name, dtype, dimensions, fill_value, values, attribute)

    dsout.close()
    dsin.close()

In [13]:
dsin = Dataset(f'/Volumes/T7/AVHRR_OI_SST/v2.1/2016/20160901120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.0-fv02.1.nc', 'r', format='NETCDF4')
dsout = Dataset(f'/Volumes/T7/new_data/AVHRR_OI_SST/2016/20160901120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.1-fv02.1.nc', 'w', format='NETCDF4')

dsout = copy_existing_variable(dsin, dsout, default_fill_value=-999)

# for variable_type, variable_value in zip(variable_types, variable_values):

#     if variable_type == 'mean' :
#         variable_name = 'mean_sst'
#         dtype = np.float32
#         attribute = {'long_name' : 'mean sst for 30 years (1981/9/1~2011/8/31)',
#                      'add_offset' : 0.0,
#                      'scale_factor' : 0.01,
#                      'valid_min' : -1200,
#                      'valid_max' : 1200,
#                      'units' : 'celcius'}

#     if variable_type == 'perc' :
#         variable_name = '90-percentile_sst'
#         dtype = np.float32
#         attribute = {'long_name' : '90-percentile of sst for 30years (1981/9/1~2011/8/31)',
#                      'add_offset' : 0.0,
#                      'scale_factor' : 0.01,
#                      'valid_min' : -1200,
#                      'valid_max' : 1200,
#                      'units' : 'celcius'}

#     if variable_type == 'cnt' :
#         variable_name = 'cnt_of_value'
#         dtype = np.int16
#         attribute = {'long_name' : 'based values counted by pixel (30,0 = normal, other = anormal)',
#                      'add_offset' : 0,
#                      'scale_factor' : 1,
#                      'valid_min' : 0,
#                      'valid_max' : 30,
#                      'units' : '-'}

#     # common attribute
#     dimensions = ('time', 'zlev', 'lat', 'lon')
#     fill_value = -999
#     values = variable_value[(month,day)]

#     dsout = create_new_variable(dsout, variable_name, dtype, dimensions, fill_value, values, attribute)

dsout.close()
dsin.close()

PermissionError: [Errno 13] Permission denied: b'/Volumes/T7/new_data/AVHRR_OI_SST/2016/20160901120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.1-fv02.1.nc'

In [ ]:
months = range(1,13)
day_lens = [31,28,31,30,31,30,31,31,30,31,30,31]

variable_types = ['mean', 'perc', 'cnt']
variable_values = [mean, perc, cnt]

for year in tqdm(range(1981,2022)):
    for month, day_len in zip(months, day_lens):
        for day in range(1,day_len+1):
            if year == 1981 and month in [1,2,3,4,5,6,7,8]:
                continue
            if year == 2021 and month == 1 and day == 16 :
                continue
            transfer_data(year, month, day, variable_types, variable_values)

  0%|          | 0/41 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/jeongseunghwan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/3k/05vd5sp53wbgspztyxv0ksfm0000gn/T/ipykernel_24627/1740563738.py", line 14, in <module>
    transfer_data(year, month, day, variable_types, variable_values)
  File "/var/folders/3k/05vd5sp53wbgspztyxv0ksfm0000gn/T/ipykernel_24627/2485455174.py", line 19, in transfer_data
    dsout = Dataset(f'/Volumes/T7/new_data/AVHRR_OI_SST/{year}/{date}120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.1-fv02.1.nc', 'w', format='NETCDF4')
  File "src/netCDF4/_netCDF4.pyx", line 2353, in netCDF4._netCDF4.Dataset.__init__
  File "src/netCDF4/_netCDF4.pyx", line 1963, in netCDF4._netCDF4._ensure_nc_success
PermissionError: [Errno 13] Permission denied: b'/Volumes/T7/new_data/AVHRR_OI_SST/1981/19810901120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.1-fv02.1.nc'

Durin

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



  File "/Users/jeongseunghwan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'PermissionError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jeongseunghwan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3364, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/Users/jeongseunghwan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3461, in run_code
    self.showtraceback(running_compiled_code=True)
  File "/Users/jeongseunghwan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2066, in showtraceback
    stb = self.InteractiveTB.structured_traceback(etype,
  File "/Users/jeongseunghwan/opt/anaconda3/lib/python3.8/site-packages/IPython

In [23]:
dsout.close()
dsin.close()

NameError: name 'dsout' is not defined

In [17]:
dsout = Dataset(f'/Volumes/T7/new_data/AVHRR_OI_SST/oisst-avhrr-v02r01.19810901.nc', 'r', format='NETCDF4')

masked_array(
  data=[[[[        nan,         nan,         nan, ...,         nan,
                   nan,         nan],
          [        nan,         nan,         nan, ...,         nan,
                   nan,         nan],
          [        nan,         nan,         nan, ...,         nan,
                   nan,         nan],
          ...,
          [-0.01212333, -0.01212333, -0.01212333, ..., -0.01212333,
           -0.01212333, -0.01212333],
          [-0.01210333, -0.01210333, -0.01210333, ..., -0.01210667,
           -0.01210333, -0.01210333],
          [-0.01211   , -0.01211   , -0.01211   , ..., -0.01211   ,
           -0.01211   , -0.01211   ]]]],
  mask=False,
  fill_value=1e+20,
  dtype=float32)